In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam


In [13]:
data_dir = './data/data'

In [15]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [17]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(640, 360),
    batch_size=32,
    class_mode='binary',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(640, 360),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1962 images belonging to 2 classes.
Found 489 images belonging to 2 classes.


In [19]:
model = Sequential()
model.add(Input(shape=(360, 640, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [20]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=20)

Epoch 1/20


C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/62 ━━━━━━━━━━━━━━━━━━━━ 11:13 48s/step - accuracy: 0.5513 - loss: 22.7296

KeyboardInterrupt: 

In [ ]:
model.save('character_model.keras')

In [ ]:
path = 'C:/Windows/Fonts/malgun.ttf'
fontprop = fm.FontProperties(fname=path, size=12)

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='훈련 정확도')
plt.plot(history.history['val_accuracy'], label='검증 정확도')
plt.title('정확도 변화', fontproperties=fontprop)
plt.xlabel('Epochs', fontproperties=fontprop)
plt.ylabel('정확도', fontproperties=fontprop)
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='훈련 손실')
plt.plot(history.history['val_loss'], label='검증 손실')
plt.title('손실 변화', fontproperties=fontprop)
plt.xlabel('Epochs', fontproperties=fontprop)
plt.ylabel('손실', fontproperties=fontprop)
plt.legend()

plt.show()

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

In [ ]:
try:
    model = load_model('character_model.keras')  # 모델 파일 이름
except Exception as e:
    print(f"모델 로드 오류: {e}")

In [ ]:
video_path = 'C:/Users/USER/Desktop/simson.mp4'
output_dir = 'C:/Users/USER/Desktop/frames'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("비디오 파일을 열 수 없습니다.")
    exit()

frame_count = 0
fps = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

In [ ]:
resized_frame = cv2.resize(frame, (360, 640))
    normalized_frame = resized_frame / 255.0
    reshaped_frame = np.reshape(normalized_frame, (1, 360, 640, 3))

    # 예측
    prediction = model.predict(reshaped_frame)
    confidence = prediction[0][0]  # 이진 분류의 경우

    # 예측된 클래스에 따라 캐릭터 이름을 매핑
    if confidence > 0.69:
        time_in_seconds = frame_count / fps
        print(f"프레임 {frame_count}: 신뢰도 {confidence:.2f}에서 인식됨")

        # 프레임 저장
        output_frame_path = os.path.join(output_dir, f'frame_{frame_count}.jpg')
        cv2.imwrite(output_frame_path, frame)
    else:
        print(f"프레임 {frame_count}: 신뢰도가 낮아 인식되지 않음, 신뢰도: {confidence:.2f}")

    frame_count += 1


In [ ]:
cap.release()
cv2.destroyAllWindows()